## LangChain OpeanAI Proxy

You may have access to a large langugage model through a HTTP Proxy like this one
https://github.com/derickson/ExpressOpenAIChatProxy

Assuming you know the URL for where it is hosted and the KEY for the proxy here is the code for using it.

your .env file will need this

```
PROXY_OPENAI_API_KEY="YOUR KEY"
PROXY_OPENAI_API_BASE="HTTPS://YOUR URL ENDING IN /v1"
```


In [ ]:
! pip install -q openai==1.14.3  langchain==0.1.13 langchain-openai==0.1.1  python_dotenv
import os
import openai
import langchain
from dotenv import load_dotenv

## suppress some warnings
import warnings, os
warnings.filterwarnings("ignore", category=DeprecationWarning, message="on_submit is deprecated.*")

FILE="Studygroup"

# workshop environment - this is where you'll enter a key
! pip install -qqq git+https://github.com/elastic/notebook-workshop-loader.git@main
from notebookworkshoploader import loader
from dotenv import load_dotenv
loader.load_remote_env(file=FILE, env_url="https://notebook-workshop-api-voldmqr2bq-uc.a.run.app")

In [ ]:
import os, secrets, requests, json
import openai
from openai import OpenAI
from requests.auth import HTTPBasicAuth

#if using the Elastic AI proxy, then generate the correct API key
if os.environ['ELASTIC_PROXY'] == "True":

    if "OPENAI_API_TYPE" in os.environ: del os.environ["OPENAI_API_TYPE"]

    #generate and share "your" unique hash
    os.environ['USER_HASH'] = secrets.token_hex(nbytes=6)
    print(f"Your unique user hash is: {os.environ['USER_HASH']}")

    #get the current API key and combine with your hash
    os.environ['OPENAI_API_KEY'] = f"{os.environ['OPENAI_API_KEY']} {os.environ['USER_HASH']}"
else:
    openai.api_type = os.environ['OPENAI_API_TYPE']
    openai.api_version = os.environ['OPENAI_API_VERSION']

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.default_model = os.environ['OPENAI_API_ENGINE']

import json
# pretty printing JSON objects
def json_pretty(input_object):
  print(json.dumps(input_object, indent=4))


import textwrap
# wrap text when printing, because colab scrolls output to the right too much
def wrap_text(text, width):
    wrapped_text = textwrap.wrap(text, width)
    return '\n'.join(wrapped_text)

def print_light_blue(text):
    print(f'\033[94m{text}\033[0m')


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=openai.api_key, base_url=openai.api_base)

llm.invoke("Hello, what is the capital of the United States?").content


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage

system_message = SystemMessage(content="""
You are an unhelpful pirate named LLM_Beard that talks like a pirate in short responses 
and ignores the questions and redirects all conversation to your love of treasure.""")
user_maessage = HumanMessage(content="What color is the sky and why?")

response = llm.invoke([system_message, user_maessage]).content

print(response)

In [ ]:
## Chat bot With memory in LangChain with OpenAI

from langchain.chains import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

import json
# pretty printing JSON objects
def json_pretty(input_object):
  print(json.dumps(input_object, indent=4))


import textwrap
# wrap text when printing, because colab scrolls output to the right too much
def wrap_text(text, width):
    wrapped_text = textwrap.wrap(text, width)
    return '\n'.join(wrapped_text)

template = """The following is a serious conversation between a human and a TV
News Anchor named Newsy McNewserson.
The Anchor provides autoritative information and commentary in short responses.
If the Anchor does not know the answer to a question,
it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
Anchor:"""

MEMORY = ConversationBufferWindowMemory(ai_prefix="Anchor", k=2)

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = LLMChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=MEMORY
)

def chatLoop():
  print(" -- Have a conversation with a TV news Anchor: ")
  print(" -- Ask this AI \"what is in the news?\" ")
  print(" -- type 'exit' when done")

  user_input = input("> ")
  while not user_input.lower().startswith("exit"):
      print( conversation.run(input=user_input) )
      print(" -- type 'exit' when done")
      user_input = input("> ")
  print("\n -- end conversation --")
     

In [ ]:
## start a new chat each time
MEMORY.clear()
## start the chat
chatLoop()
     